# fastai Sudoku

> "Improve our fastai data preparation understanding by playing Sudoku"

- toc: true
- branch: master
- badges: true
- comments: true
- author: Craig Stanton
- hide: false
- categories: [fastai]

Use a Sudoku puzzle to learn more about the fastai `DataBlock`, `Datasets`, `DataLoaders` and `TfmdDL` objects.

Why even bother with this? Jeremy's advice is to build models as quick as you can. However I often find myself tripping at the first hurdle many times - **the data preparation stage**. It doesn't matter how many times I read about `DataBlock`, `TfmdDL`, and `DataLoaders`, there is no substitute for actually using the libraries. So why not outline a game that requires you to build and solve the puzzle by using the same tools that you need to structure the data for a fastai `Learner` (which is where the actual training magic happens).

Additionally, I really struggle at times to read the fastai code - it contains so many Python tricks that I am not always familiar with. Therefore this type of exercise forces me to interrogate the code in order to write working DataBlock functions.

## Why Sudoku?

Besides the fact that most people know the game, I needed something that was 2-dimensional because that is how our raw ML data is usually represented (even if we are working with images, its helpful to *think* of the input data in a tabular structure, where each training item is represented as a row, and one column is the independent variable and while another is the dependent variable).

### Setup

In [ ]:
#@title
!pip install py-sudoku==1.0.3
from fastai.text.all import *
from fastai.vision.all import *
from sudoku import Sudoku
from functools import wraps
from typing import Union, Iterable
from collections.abc import Collection
import ipywidgets as widgets
import requests, pprint
from IPython.display import HTML

class FastSudoku:
    """
    Learn how to use the fastai DataBlock, TfmdDL, Datasets, and DataLoaders transforms and callbacks by creating and solving a sudoku puzzle
    """
    
    def __init__(self, difficulty: float = 0.25, seed: int = 527, data_dir: Path = Path(".")):
        self.puzzle = Sudoku(3, seed=seed).difficulty(difficulty)
        self.solved = self.puzzle.solve().board
        pd.DataFrame(self.puzzle.board).to_csv(data_dir/"raw_data.csv", index=False)
        
    def __repr__(self):
        return f"Puzzle created"
    
    @staticmethod
    def np2list(x): return None if np.isnan(x) else int(x)
    
    def check(self, dls):
        """
        Unpack the dataloaders output, convert to int and str
        """
        holder = []
        for dl in dls:
            holder+=dl
        self.preds = [list(map(self.np2list,j)) for j in [i[0] for i in holder]]
        if all(j for j in [self.solved[i] == x for i, x in enumerate(self.preds)]):
            
            print("\n\nYes you are a fastai...and sudoku...whiz!\n\n")
        else:
            print("\n\nNot quite. Check out your puzzle below and try again!\n\n")
            # print the current board with any guesses
            Sudoku(3, 3, board=self.preds).show()

fast_sudoku_answers = requests.get("https://gist.githubusercontent.com/stantonius/ca95d88dcd0085b12a302f64b326caf8/raw/68b676048138fc8096157176b411b677a98a34ec/fast_sudoku_answers.json")

### Getting Started

The first thing to do is to set the puzzle **difficulty** (a float between 0 and 1) and the **seed** (any integer). Note that the seed ensures the reproducability of the same Sudoku board - therefore, in order to practice a second time you will want to change this value)

In [ ]:
fs = FastSudoku(difficulty = 0.2, seed = 92)

In [ ]:
fs.puzzle.show()

### Challenge 1 - Create a `Datasets` object using the `DataBlock` API

In the current directory we have a generated file `raw_data.csv` which contains the unsolved Sudoku puzzle data. 

**Objective**: Using the `DataBlock` API, pass as many function arguments (ie. `get_items`, `batch_tfms`, `get_y`, etc.) as possible so that you create a `Datasets` object that contains the original Sudoku values *along with an additional `y-value` column*.  

**Instructions**:

* Grab the `raw_data.csv` puzzle and create a `DataBlock`
* You should try and use *as many* of the functions below as DataBlock arguments - not necessarily all, but as many as you can/wish. The point of this is not to be the most efficient or practical way of creating a `DataBlock` but rather to *understand what each argument function argument does*.

*Tips*:
1. Don't be afraid to comment out lines to see how the absence of functions changes the output
2. Use print statements to track the output of each function
3. Read the fastai code for the functions and classes you are unfamiliar with.

**Question**: the objective states to create a new `y-value` column. What should this column contain? Why? 

*Click on the `Show answer` buttons below to reveal the answers*

In [ ]:
#@title
button = widgets.Button(description="Show answer")
output = widgets.Output()

def show_answer(b):
    with output:
        print(fast_sudoku_answers.json()["y_col"])

button.on_click(callback=show_answer)
display(button, output)

In [ ]:
#@title
button = widgets.Button(description="Show answer")
output = widgets.Output()

def show_answer(b):
    with output:
        print(fast_sudoku_answers.json()["why_y"])

button.on_click(callback=show_answer)
display(button, output)

#### Define our `DataBlock` functional arguments

In [ ]:
def get_items():
    pass

def get_x():
    pass

def get_y():
    pass
    

#################
# Predefined
#################

def splitter(a):
    # In this exercise, we don't need train and validation sets
    # Therefore this is effectively a dummy function in this specific circumstance
    # But never forget about Splitter because it is such an important concept!
    return [list(range(9)),]

In [ ]:
dblock = DataBlock(
    get_items=get_items,
    get_x=get_x,
    get_y=get_y,
    splitter=splitter
)

**Question**: why did we not need to provide any `blocks` arguments?

In [ ]:
#@title
button = widgets.Button(description="Show answer")
output = widgets.Output()

def show_answer(b):
    with output:
        print(fast_sudoku_answers.json()["no_blocks"])

button.on_click(callback=show_answer)
display(button, output)

**Question**: why did we not supply any `batch_tfms` or `item_tfms`?

In [ ]:
#@title
button = widgets.Button(description="Show answer")
output = widgets.Output()

def show_answer(b):
    with output:
        print(fast_sudoku_answers.json()["no_tfms"])

button.on_click(callback=show_answer)
display(button, output)

#### Check our Dataset

In [ ]:
dsets = dblock.datasets("raw_data.csv");

In [ ]:
fs.puzzle.show()

dsets.items

### Challenge 2 - Use `TfmdDL` to solve the Sudoku puzzle

> Why are we using `TfmdDL` here? If you look into the fastai code, any time you create a `DataLoaders`, the `TfdmDL` class is ultimately called.

**Objective:** Use the `TfmdDL` callbacks to modify the `Datasets` you just created to solve the Sudoku board

**Instructions**:
* Test your DataLoaders object against the puzzle, you can use the `fs.check(dls)` method
* You should try and use *as many* of the callback functions below - not necessarily all, but as many as you can/wish. The point of this is not to be the most efficient or practical way of creating a `DataLoaders` but rather to understand what each function argument does.

*Tips*:
1. Use **helper function(s)** that are called within other functions to insert your Sudoku responses into the row
2. Don't be afraid to comment out lines to see how the absence of functions changes the output
3. Use print statements to track the output of each function
4. Read the fastai code for the functions and classes you are unfamiliar with.

In [ ]:
# show the Sudoku puzzle again
fs.puzzle.show()

In [ ]:
def before_iter():
    pass

def after_item(a):
    pass

def before_batch(a):
    pass

def after_iter(cls_name):
    pass

def create_batches(a):
    pass

def create_item(a):
    pass

def custom_collate(a):
    pass

def create_batch(a):
    pass

def after_batch(a):
    pass

In [ ]:
dls = TfmdDL(
    dsets,
    bs=2,   # change this value to see its effects
    before_iter=before_iter,
    after_item=after_item,
    before_batch=before_batch,
    after_iter=after_iter,
    create_item=create_item,
    create_batch=create_batch,
    after_batch=after_batch,
    create_batches=create_batches,
    shuffle=False # change to see the impact
)


Use the `fs.check()` function to check your answers.

In [ ]:
fs.check(dls)

# Notes

Add your notes here